# IR Lab SoSe 2024: Baseline Retrieval System

This jupyter notebook serves as baseline retrieval system that you can try to improve upon.
We will use the a corpus of scientific papers (title + abstracts) from the fields of information retrieval and natural language processing (the [IR Anthology](https://ir.webis.de/anthology/) and the [ACL Anthology](https://aclanthology.org/)). This serves Jupyter notebook only serves as retrieval system, i.e., it gets a set of information needs (topics) and a corpus as input and produces a run file as output. Please do evaluations in a new dedicated notebook.

### Step 1: Import Libraries

We will use [tira](https://www.tira.io/), an information retrieval shared task platform, for loading the (pre-built) retrieval index and [ir_dataset](https://ir-datasets.com/) to subsequently build a retrieval system with [PyTerrier](https://github.com/terrier-org/pyterrier), an open-source search engine.

Building your own index can be already one way that you can try to improve upon this baseline (if you want to focus on creating good document representations). Other ways could include reformulating queries or tuning parameters or building better retrieval pipelines.

In [94]:
# You only need to execute this cell if you are using Google Golab.
# If you use GitHub Codespaces, everything is already installed.
!pip3 install tira ir-datasets python-terrier

In [1]:
# Imports
from tira.third_party_integrations import ensure_pyterrier_is_loaded, persist_and_normalize_run
from tira.rest_api_client import Client
import pyterrier as pt
# stopword imports
import nltk
from nltk.corpus import stopwords
import spacy
from sklearn.feature_extraction.text import ENGLISH_STOP_WORDS
# lemmatizer imports
import pandas as pd
pd.set_option('display.max_colwidth', 0)
# sklearn import
from sklearn.ensemble import RandomForestRegressor

# Create a REST client to the TIRA platform for retrieving the pre-indexed data.
ensure_pyterrier_is_loaded()
tira = Client()
# spacy model
!python -m spacy download en_core_web_sm

from jnius import autoclass
'''
if not pt.started():
    pt.init(boot_packages=['mam10eks:custom-terrier-token-processing:0.0.1'])
    from jnius import autoclass
'''

PyTerrier 0.10.0 has loaded Terrier 5.8 (built by craigm on 2023-11-01 18:05) and terrier-helper 0.0.8

No etc/terrier.properties, using terrier.default.properties for bootstrap configuration.


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.8/12.8 MB 5.7 MB/s eta 0:00:0000:0100:01m
✔ Download and installation successful
You can now load the package via spacy.load('en_core_web_sm')


"\nif not pt.started():\n    pt.init(boot_packages=['mam10eks:custom-terrier-token-processing:0.0.1'])\n    from jnius import autoclass\n"

### Step 2: Stopword Removal

In [2]:
# download stopwords
nltk.download('stopwords')

# Generate custom stopword list
nltk_stopwords = set(stopwords.words('english'))
nlp = spacy.load("en_core_web_sm")
spacy_stopwords = set(nlp.Defaults.stop_words)
sklearn_stopwords = set(ENGLISH_STOP_WORDS)
combined_stopwords = set.union(nltk_stopwords, spacy_stopwords, sklearn_stopwords)

# output to verify stopwords
print('a quick look at the stopwords')
print(combined_stopwords)

## Create and save stopword file
# file_path = "../custom-stopwords/custom_stopwords.txt"
file_path = "./custom_stopwords.txt"

with open(file_path, 'w+') as file:
    for element in combined_stopwords:
        file.write(element + "\n")

# Set property for stopword file in PyTerrier
# pt.set_property('stopwords.filename', '../custom-stopwords/custom_stopwords.txt')
pt.set_property('stopwords.filename', './custom_stopwords.txt')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


a quick look at the stopwords
{'whereby', 'together', 'hasnt', 'five', "shouldn't", 'whole', 'just', 'off', 'sincere', 'my', 'side', 'this', 'whenever', 'ours', 'put', 'yourself', 'mustn', 'have', 'go', 'whereupon', 'down', '’re', 'seems', 'amount', 'ltd', 'through', 'sometime', 'via', "mightn't", 'ever', 'aren', 'became', 'too', 'was', 'becoming', 'having', 'until', 'part', "wouldn't", 'someone', 'already', '‘ve', 'm', 'doing', 't', 'it', 'detail', 'doesn', 'within', 'quite', 'latterly', 'nothing', 'another', 'hasn', 'afterwards', 'hers', 'am', 'less', 'take', 'mill', 'two', 'shouldn', 'alone', 'everyone', 'made', 'what', 'over', "didn't", 'say', 'therefore', '’s', '‘s', "aren't", 'mostly', 'wherein', 'thru', 'perhaps', 'eleven', 'con', 'you', "needn't", 'been', 'which', 'll', 'while', 'under', 'one', "won't", 'mightn', 'ca', 've', 'she', 'd', 'ma', "hasn't", 'latter', 'anywhere', "should've", 'why', 'will', 'hundred', 'noone', 'hence', 'towards', 'either', 'sixty', 'front', 'weren', 

### Step 3: Load the Dataset

In [3]:
print('Loading Dataset...')
# This line creates an IRDSDataset object and registers it under the name provided as an argument.
pt_dataset = pt.get_dataset('irds:ir-lab-sose-2024/ir-acl-anthology-20240504-training')
print('Dataset loaded.')

Loading Dataset...
Dataset loaded.


### Step x: Lemmatizer

In [4]:
# basic lemmatizer implementation
def lemmatize(t):
    lemmatizer = autoclass("org.terrier.terms.StanfordLemmatizer")()
    return lemmatizer.stem(t)

# porterStemmer
def stem(t):
    stemmer = autoclass("org.terrier.terms.PorterStemmer")()
    return stemmer.stem(t)

# lemurKrovetzStemmer
def stem_krovetz(t):
    stemmer = autoclass("org.terrier.terms.LemurKrovetzStemmer")()
    return stemmer.stem(t)

### Step 4: Index Building

In [5]:
print('Building Index...')

def create_index(pt_dataset, stopwords):
    # added lemmatizer
    indexer = pt.IterDictIndexer("/tmp/index", overwrite=True, meta={'docno': 100, 'text': 20480}, stopwords=stopwords, stemmer='PorterStemmer') # stemmer needs to be set
    index_ref = indexer.index(pt_dataset)
    return pt.IndexFactory.of(index_ref)

# custom stopword index
index = create_index(pt_dataset.get_corpus_iter(), combined_stopwords)

# A (pre-built) PyTerrier index loaded from TIRA
# index = tira.pt.index('ir-lab-sose-2024/tira-ir-starter/Index (tira-ir-starter-pyterrier)', pt_dataset)
print('Index created.')

Building Index...


ir-lab-sose-2024/ir-acl-anthology-20240504-training documents: 100%|██████████| 126958/126958 [00:13<00:00, 9713.06it/s] 


Index created.


### Step 5: Define the Retrieval Pipeline

We will define a BM25 retrieval pipeline as baseline. For details, see:

- [https://pyterrier.readthedocs.io](https://pyterrier.readthedocs.io)
- [https://github.com/terrier-org/ecir2021tutorial](https://github.com/terrier-org/ecir2021tutorial)

In [11]:
# definition of pipelines
bm25 = pt.BatchRetrieve(index, wmodel="BM25")
tf = pt.BatchRetrieve(index, wmodel="Tf")
pl2 = pt.BatchRetrieve(index, wmodel="PL2")

# bo1 query expansion
bo1 = pt.rewrite.Bo1QueryExpansion(index)
bm25_expand = bm25 >> bo1 >> bm25

# reranking
pipeline = bm25_expand >> (tf ** pl2)

# sklearn regressor
# rf = RandomForestRegressor(n_estimators=5)
# rf_pipe = pipeline >> pt.ltr.apply_learned_model(rf)
# rf_pipe.fit(pt_dataset.get_topics(), pt_dataset.get_qrels())

### Step 6: Create the Run


In [7]:
print('First, we have a short look at the first three topics:')
pt_dataset.get_topics('text').head(3)

First, we have a short look at the first three topics:


,qid,query
0,1,retrieval system improving effectiveness
1,2,machine learning language identification
2,3,social media detect self harm


In [8]:
print('Create run')
run = pipeline(pt_dataset.get_topics('text'))
print('Done. Here are the first 10 entries of the run')
run.head(10)

Create run
Done. Here are the first 10 entries of the run


,qid,docid,docno,score,query,features,rank
0,1,94858,2004.cikm_conference-2004.47,0.6,retrieval system improving effectiveness,"[22.0, 9.600797845496768]",15
1,1,125137,1989.ipm_journal-ir0volumeA25A4.2,0.6,retrieval system improving effectiveness,"[4.0, 9.618979393386638]",16
2,1,84876,2016.ntcir_conference-2016.90,0.8,retrieval system improving effectiveness,"[14.0, 8.401517143790793]",2
3,1,5868,W05-0704,0.0,retrieval system improving effectiveness,"[15.0, 9.282615310408456]",52
4,1,125817,2005.ipm_journal-ir0volumeA41A5.11,0.8,retrieval system improving effectiveness,"[15.0, 7.8862767550893285]",3
5,1,126826,2007.tois_journal-ir0volumeA26A1.4,0.6,retrieval system improving effectiveness,"[16.0, 8.818200605377877]",17
6,1,94415,2008.cikm_conference-2008.183,0.8,retrieval system improving effectiveness,"[18.0, 8.10588972959852]",4
7,1,124801,2006.ipm_journal-ir0volumeA42A3.2,0.4,retrieval system improving effectiveness,"[17.0, 7.677016688150646]",24
8,1,81840,2006.sigirconf_conference-2006.103,0.0,retrieval system improving effectiveness,"[8.0, 7.669430024361412]",53
9,1,82472,1998.sigirconf_conference-98.15,0.8,retrieval system improving effectiveness,"[12.0, 7.7792092708026255]",5


### Step 7: Persist the run file for subsequent evaluations

The output of a prototypical retrieval system is a run file. This run file can later (optimally in a different notebook) be statistically evaluated.

In [9]:
persist_and_normalize_run(run, system_name='bm25-knowledge-knights', output_file='../runs')

Done. run file is stored under "../runs/run.txt".


### evaluation

In [10]:
'''
bm25 = pt.io.read_results('../runs/run.txt')

bm25_baseline = tira.pt.from_submission('ir-benchmarks/tira-ir-starter/BM25 (tira-ir-starter-pyterrier)', pt_dataset)
sparse_cross_encoder = tira.pt.from_submission('ir-benchmarks/fschlatt/sparse-cross-encoder-4-512', pt_dataset)
rank_zephyr = tira.pt.from_submission('workshop-on-open-web-search/fschlatt/rank-zephyr', pt_dataset)

pt.Experiment(
    [pipeline, bm25_baseline, sparse_cross_encoder, rank_zephyr],
    pt_dataset.get_topics(),
    pt_dataset.get_qrels(),
    ["ndcg_cut.10", "recip_rank", "recall_100"],
    names=["BM25 (Own)", "BM 25 (Baseline)", "Sparse Cross Encoder", "RankZephyr"]
)
'''

There are multiple query fields available: ('text', 'title', 'query', 'description', 'narrative'). To use with pyterrier, provide variant or modify dataframe to add query column.


,name,ndcg_cut.10,recip_rank,recall_100
0,BM25 (Own),0.905453,0.970588,0.816418
1,BM 25 (Baseline),0.374041,0.579877,0.601333
2,Sparse Cross Encoder,0.366460,0.612980,0.601333
3,RankZephyr,0.347070,0.568413,0.601333
